In [ ]:
from experiments.attn.victory import *

In [ ]:
losses = run_trial(common.ConvContextModel, boardsize=9, n_layers=4, D=32)

In [ ]:
s = """995    0.019779
996    0.017397
997    0.012021
998    0.022778
999    0.005439"""
pd.Series(dict([l.split() for l in s.splitlines()])).pipe(pd.to_numeric).apply(np.log10).mean()

In [ ]:
losses.apply(np.log10).tail(10).mean()

In [ ]:
losses.apply(np.log10).tail(10).mean()

In [ ]:
df = load('*angry-alarm*')

In [ ]:
finals = df.tail(10).apply(np.log10).mean().reset_index().rename(columns={0: 'val'})

fig, axes = plt.subplots(1, 2)

ax = axes[0]
finals.query('depth == 16').groupby(['width', 'boardsize']).val.mean().unstack('width').plot(marker='o', ax=ax)

ax = axes[1]
finals.query('width == 64').groupby(['depth', 'boardsize']).val.mean().unstack('depth').plot(marker='o', ax=ax)

for ax in axes:
    ax.set_title('"victory" toy loss')
    ax.grid(True)

fig.set_size_inches(15, 6)

In [ ]:
from ptflops import get_model_complexity_info

model = common.FCModel(common.PosActions, 9, 256, 16).cuda()
obs = torch.zeros((1, 9, 9, 2)).cuda()

macs, params = get_model_complexity_info(model, (9, 9, 2), as_strings=False, print_per_layer_stat=False)

In [ ]:
model = common.FCModel
for boardsize in (3, 5, 7, 9):
    for depth in [1, 2, 4, 8, 16]:
        for width in [2, 4, 8, 16, 32, 64]:
            print(boardsize, depth, width)
            m = common.FCModel(common.PosActions, boardsize, 256, 16).cuda()
            macs, params = get_model_complexity_info(model, (9, 9, 2), as_strings=False, print_per_layer_stat=False)

In [ ]:
model = common.ConvModel

run = '*angry-alarm*'
i = runs.info(run)
assert model.__name__ == i['model']

In [ ]:
for n, f in i['_files'].items():
    boardsize = f['boardsize']
    depth = f['depth']
    width = f['width']
    m = model(common.PosActions, boardsize, width, depth).cuda()
    macs, params = get_model_complexity_info(m, (boardsize, boardsize, 2), as_strings=False, print_per_layer_stat=False)
    
    with runs.update(run) as ri:
        ri['_files'][n]['n_macs'] = macs
        ri['_files'][n]['n_params'] = params